In [12]:
import overpy
import shapefile
import os

# Initialize the Overpass API
api = overpy.Overpass()

# List of administrative levels to query
admin_levels = [4, 6]

# Function to save Overpass API result to a shapefile
def save_to_shapefile(result, file_path):
    with shapefile.Writer(file_path) as shp:
        shp.field('id', 'N')
        shp.field('name', 'C')
        
        for way in result.ways:
            nodes = [(float(node.lon), float(node.lat)) for node in way.nodes]
            if len(nodes) > 2:  # Ensuring the geometry is a polygon
                shp.poly([nodes])
                shp.record(id=way.id, name=way.tags.get('name', ''))

# Create directory to save shapefiles
if not os.path.exists('shapefiles'):
    os.makedirs('shapefiles')

# Query and save each administrative level
for level in admin_levels:
    query = f"""
    [out:json];
    area["ISO3166-1"="FR"][admin_level=2];
    (
      relation["admin_level"="{level}"](area);
    );
    out body;
    >;
    out skel qt;
    """
    try:
        result = api.query(query)
        save_to_shapefile(result, f'shapefiles/france_admin_regions_level_{level}')
        print(f'Shapefiles for admin level {level} saved successfully.')
    except Exception as e:
        print(f'Error for admin level {level}: {e}')

print("All shapefiles saved successfully in the 'shapefiles' directory.")


Error for admin level 4: IncompleteRead(0 bytes read)


KeyboardInterrupt: 

In [11]:
import ee
import geopandas as gpd
from shapely.geometry import shape

# Initialize the Google Earth Engine (assuming the user is already authenticated)
ee.Initialize()

# Define the feature collection for France administrative regions
france_admin_regions = ee.FeatureCollection("FAO/GAUL/2015/level1")\
    .filter(ee.Filter.eq('ADM0_NAME', 'France'))

# Convert the feature collection to GeoJSON
geojson = france_admin_regions.getInfo()

# Extract valid geometries (e.g., polygons)
valid_features = []
for feature in geojson['features']:
    geom = shape(feature['geometry'])
    if geom.geom_type == 'Polygon' or geom.geom_type == 'MultiPolygon':
        valid_features.append(feature)

# Convert valid GeoJSON features to GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(valid_features)

# Create directory to save shapefiles
if not os.path.exists('shapefiles'):
    os.makedirs('shapefiles')

# Save GeoDataFrame to shapefile
gdf.to_file('shapefiles/france_admin_regions.shp')

print("Shapefiles saved successfully in the 'shapefiles' directory.")


Shapefiles saved successfully in the 'shapefiles' directory.


In [13]:
import networkx as nx

G = nx.DiGraph()

# 1. Load France administrative regions shapefile
G.add_node("france_admin_shp_path", node_type="data", data_path="shapefiles/france_admin_regions.shp", description="Path to France administrative regions shapefile")
G.add_node("load_france_admin_shp", node_type="operation", description="Load France administrative regions shapefile")
G.add_edge("france_admin_shp_path", "load_france_admin_shp")
G.add_node("france_admin_gdf", node_type="data", data_path="", description="GeoDataFrame of France administrative regions")
G.add_edge("load_france_admin_shp", "france_admin_gdf")

# 2. Load human mobility data
G.add_node("human_mobility_data_path", node_type="data", data_path="data/human_mobility_2020.csv", description="Path to human mobility data for 2020")
G.add_node("load_human_mobility_data", node_type="operation", description="Load human mobility data for 2020")
G.add_edge("human_mobility_data_path", "load_human_mobility_data")
G.add_node("human_mobility_df", node_type="data", data_path="", description="DataFrame of human mobility data for 2020")
G.add_edge("load_human_mobility_data", "human_mobility_df")

# 3. Calculate monthly change rates based on January 2020
G.add_node("calculate_monthly_change_rates", node_type="operation", description="Calculate monthly change rates based on January 2020")
G.add_edge("human_mobility_df", "calculate_monthly_change_rates")
G.add_node("monthly_change_rates_df", node_type="data", data_path="", description="DataFrame of monthly change rates")
G.add_edge("calculate_monthly_change_rates", "monthly_change_rates_df")

# 4. Join change rates to GeoDataFrame
G.add_node("join_change_rates_to_gdf", node_type="operation", description="Join change rates to GeoDataFrame")
G.add_edge("monthly_change_rates_df", "join_change_rates_to_gdf")
G.add_edge("france_admin_gdf", "join_change_rates_to_gdf")
G.add_node("france_admin_change_gdf", node_type="data", data_path="", description="GeoDataFrame of France administrative regions with change rates")
G.add_edge("join_change_rates_to_gdf", "france_admin_change_gdf")

# 5. Generate choropleth maps for each month
G.add_node("generate_choropleth_maps", node_type="operation", description="Generate choropleth maps for each month")
G.add_edge("france_admin_change_gdf", "generate_choropleth_maps")
G.add_node("monthly_choropleth_maps", node_type="data", data_path="", description="List of choropleth maps for each month")
G.add_edge("generate_choropleth_maps", "monthly_choropleth_maps")

# 6. Generate line chart for monthly change rate trends
G.add_node("generate_line_chart", node_type="operation", description="Generate line chart for monthly change rate trends")
G.add_edge("monthly_change_rates_df", "generate_line_chart")
G.add_node("line_chart_trends", node_type="data", data_path="", description="Line chart for monthly change rate trends")
G.add_edge("generate_line_chart", "line_chart_trends")

# Save the network to GraphML format
nx.write_graphml(G, "Resident_at_risk_counting.graphml")


In [20]:
import sys
sys.path.append(r"C:\Users\Sukhvansh Jain\Desktop\IIT Study Material\Projects\ISRO Hackathon\LLM-Geo")
import helper
G = nx.read_graphml("Resident_at_risk_counting.graphml")  
nt = helper.show_graph(G)
html_name = os.path.join(os.getcwd(), 'Resident_at_risk_counting' + '.html')  
# HTML file should in the same directory. See:
# https://stackoverflow.com/questions/65564916/error-displaying-pyvis-html-inside-jupyter-lab-cell
nt.show(name=html_name)

c:\Users\Sukhvansh Jain\Desktop\IIT Study Material\Projects\ISRO Hackathon\LLM-Geo\testing\Resident_at_risk_counting.html


In [22]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from io import StringIO

def Load_france_admin_shp(shp_path="shapefiles/france_admin_regions.shp"):
    """
    Load France administrative regions shapefile into a GeoDataFrame.
    shp_path: Path to the shapefile
    Returns: GeoDataFrame of France administrative regions
    """
    france_admin_gdf = gpd.read_file(shp_path)
    return france_admin_gdf

def Load_human_mobility_data(csv_path="data/human_mobility_2020.csv"):
    """
    Load human mobility data from a CSV file into a DataFrame.
    csv_path: Path to the human mobility data CSV file
    Returns: DataFrame of human mobility data for 2020
    """
    human_mobility_df = pd.read_csv(csv_path)
    return human_mobility_df

def Calculate_monthly_change_rates(human_mobility_df):
    """
    Calculate monthly change rates of human mobility based on January 2020.
    human_mobility_df: DataFrame of human mobility data for 2020
    Returns: DataFrame with monthly change rates
    """
    # Ensure the month column is in string format and the date column is in datetime format
    human_mobility_df['month'] = human_mobility_df['month'].astype(str)
    human_mobility_df['date'] = pd.to_datetime(human_mobility_df['date'])
    
    # Compute change rates
    base_january = human_mobility_df[human_mobility_df['month'] == '2020-01']['mobility'].mean()
    human_mobility_df['change_rate'] = (human_mobility_df['mobility'] - base_january) / base_january * 100
    
    # Pivot data to have months as columns
    monthly_change_rates_df = human_mobility_df.pivot(index='region', columns='month', values='change_rate').reset_index()
    return monthly_change_rates_df

def Join_change_rates_to_gdf(france_admin_gdf, monthly_change_rates_df):
    """
    Join the change rates DataFrame to the GeoDataFrame.
    france_admin_gdf: GeoDataFrame of France administrative regions
    monthly_change_rates_df: DataFrame with monthly change rates
    Returns: GeoDataFrame of France administrative regions with change rates
    """
    # Convert region column to string for merging
    france_admin_gdf['region'] = france_admin_gdf['region'].astype(str)
    monthly_change_rates_df['region'] = monthly_change_rates_df['region'].astype(str)
    
    # Merge DataFrame with GeoDataFrame
    france_admin_change_gdf = france_admin_gdf.merge(monthly_change_rates_df, on='region')
    return france_admin_change_gdf

def Generate_choropleth_maps(france_admin_change_gdf):
    """
    Generate choropleth maps for each month and save them as files.
    france_admin_change_gdf: GeoDataFrame of France administrative regions with change rates
    Returns: List of file paths for the choropleth maps
    """
    months = [str(month).zfill(2) for month in range(1, 13)]
    file_paths = []
    
    for month in months:
        fig, ax = plt.subplots(1, 1, figsize=(15, 10))
        france_admin_change_gdf.plot(column=month, cmap='coolwarm', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
        ax.set_title(f'Mobility Change Rate for {month}/2020')
        ax.set_axis_off()
        
        file_path = f'maps/choropleth_{month}.png'
        plt.savefig(file_path, bbox_inches='tight')
        plt.close(fig)
        
        file_paths.append(file_path)
    
    return file_paths

def Generate_line_chart(monthly_change_rates_df):
    """
    Generate a line chart showing monthly change rate trends for each administrative region.
    monthly_change_rates_df: DataFrame with monthly change rates
    Returns: File path to the line chart image
    """
    plt.figure(figsize=(15, 10))
    
    # Set index to 'region' for easier plotting
    monthly_change_rates_df.set_index('region').T.plot(kind='line')
    
    plt.title('Monthly Change Rate Trends of Human Mobility')
    plt.xlabel('Month')
    plt.ylabel('Change Rate (%)')
    plt.legend(title='Region')
    plt.grid(True)
    
    file_path = 'charts/monthly_change_rate_trends.png'
    plt.savefig(file_path, bbox_inches='tight')
    plt.close()
    
    return file_path


In [24]:
import ee
import pandas as pd

def authenticate_and_initialize():
    """
    Authenticate and initialize Google Earth Engine.
    """
    ee.Authenticate()  # Authenticate GEE access
    ee.Initialize()    # Initialize GEE

def download_human_mobility_data():
    """
    Download human mobility data from Google Earth Engine and save it as a CSV file.
    """
    # Example dataset ID; replace with the actual dataset ID or query
    dataset_id = 'YOUR_DATASET_ID'
    
    # Define the time range and other parameters for the data query
    time_range = ee.DateRange('2020-01-01', '2020-12-31')
    
    # Load dataset from GEE
    dataset = ee.ImageCollection(dataset_id).filterDate(time_range)
    
    # Convert to pandas DataFrame
    def image_to_dict(image):
        return ee.batch.Export.image.toDrive(image, description='export', scale=30, region=region)

    # Export to Google Drive and download to local
    export_task = ee.batch.Export.image.toDrive(
        image=dataset.mean(),  # Aggregate or process as needed
        description='human_mobility_2020',
        folder='GEE_exports',
        fileNamePrefix='human_mobility_2020',
        scale=30
    )
    
    export_task.start()
    
    print("Download initiated. Check your Google Drive for the file.")

# Authenticate and initialize GEE
authenticate_and_initialize()

# Download data
download_human_mobility_data()


EEException: ImageCollection.load: ImageCollection asset 'YOUR_DATASET_ID' not found (does not exist or caller does not have access).

In [23]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

def Load_france_admin_shp(shp_path="shapefiles/france_admin_regions.shp"):
    """
    Load France administrative regions shapefile into a GeoDataFrame.
    shp_path: Path to the shapefile
    Returns: GeoDataFrame of France administrative regions
    """
    france_admin_gdf = gpd.read_file(shp_path)
    return france_admin_gdf

def Load_human_mobility_data(csv_path="data/human_mobility_2020.csv"):
    """
    Load human mobility data from a CSV file into a DataFrame.
    csv_path: Path to the human mobility data CSV file
    Returns: DataFrame of human mobility data for 2020
    """
    human_mobility_df = pd.read_csv(csv_path)
    return human_mobility_df

def Calculate_monthly_change_rates(human_mobility_df):
    """
    Calculate monthly change rates of human mobility based on January 2020.
    human_mobility_df: DataFrame of human mobility data for 2020
    Returns: DataFrame with monthly change rates
    """
    # Ensure the month column is in string format and the date column is in datetime format
    human_mobility_df['month'] = human_mobility_df['month'].astype(str)
    human_mobility_df['date'] = pd.to_datetime(human_mobility_df['date'])
    
    # Compute change rates
    base_january = human_mobility_df[human_mobility_df['month'] == '2020-01']['mobility'].mean()
    human_mobility_df['change_rate'] = (human_mobility_df['mobility'] - base_january) / base_january * 100
    
    # Pivot data to have months as columns
    monthly_change_rates_df = human_mobility_df.pivot(index='region', columns='month', values='change_rate').reset_index()
    return monthly_change_rates_df

def Join_change_rates_to_gdf(france_admin_gdf, monthly_change_rates_df):
    """
    Join the change rates DataFrame to the GeoDataFrame.
    france_admin_gdf: GeoDataFrame of France administrative regions
    monthly_change_rates_df: DataFrame with monthly change rates
    Returns: GeoDataFrame of France administrative regions with change rates
    """
    # Convert region column to string for merging
    france_admin_gdf['region'] = france_admin_gdf['region'].astype(str)
    monthly_change_rates_df['region'] = monthly_change_rates_df['region'].astype(str)
    
    # Merge DataFrame with GeoDataFrame
    france_admin_change_gdf = france_admin_gdf.merge(monthly_change_rates_df, on='region')
    return france_admin_change_gdf

def Generate_choropleth_maps(france_admin_change_gdf):
    """
    Generate choropleth maps for each month and save them as files.
    france_admin_change_gdf: GeoDataFrame of France administrative regions with change rates
    Returns: List of file paths for the choropleth maps
    """
    months = [str(month).zfill(2) for month in range(1, 13)]
    file_paths = []
    
    for month in months:
        fig, ax = plt.subplots(1, 1, figsize=(15, 10))
        france_admin_change_gdf.plot(column=month, cmap='coolwarm', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)
        ax.set_title(f'Mobility Change Rate for {month}/2020')
        ax.set_axis_off()
        
        file_path = f'maps/choropleth_{month}.png'
        plt.savefig(file_path, bbox_inches='tight')
        plt.close(fig)
        
        file_paths.append(file_path)
    
    return file_paths

def Generate_line_chart(monthly_change_rates_df):
    """
    Generate a line chart showing monthly change rate trends for each administrative region.
    monthly_change_rates_df: DataFrame with monthly change rates
    Returns: File path to the line chart image
    """
    plt.figure(figsize=(15, 10))
    
    # Set index to 'region' for easier plotting
    monthly_change_rates_df.set_index('region').T.plot(kind='line')
    
    plt.title('Monthly Change Rate Trends of Human Mobility')
    plt.xlabel('Month')
    plt.ylabel('Change Rate (%)')
    plt.legend(title='Region')
    plt.grid(True)
    
    file_path = 'charts/monthly_change_rate_trends.png'
    plt.savefig(file_path, bbox_inches='tight')
    plt.close()
    
    return file_path

def assemble_solution():
    # Load data
    france_admin_gdf = Load_france_admin_shp()
    human_mobility_df = Load_human_mobility_data()
    
    # Calculate change rates
    monthly_change_rates_df = Calculate_monthly_change_rates(human_mobility_df)
    
    # Join change rates with GeoDataFrame
    france_admin_change_gdf = Join_change_rates_to_gdf(france_admin_gdf, monthly_change_rates_df)
    
    # Generate choropleth maps
    choropleth_files = Generate_choropleth_maps(france_admin_change_gdf)
    
    # Generate line chart
    line_chart_file = Generate_line_chart(monthly_change_rates_df)
    
    return choropleth_files, line_chart_file

# Execute the solution assembly
assemble_solution()


FileNotFoundError: [Errno 2] No such file or directory: 'data/human_mobility_2020.csv'

In [2]:
import overpy
from shapely.geometry import Point
from shapely.ops import transform
from pyproj import Transformer

# Initialize the Overpass API
api = overpy.Overpass()

# Query for Kalka
kalka_query = """
node["name"="Kalka"];
out body;
"""
kalka_result = api.query(kalka_query)
kalka_node = kalka_result.nodes[0]
kalka_location = (kalka_node.lat, kalka_node.lon)

# Query for Delhi
delhi_query = """
node["name"="Delhi"];
out body;
"""
delhi_result = api.query(delhi_query)
delhi_node = delhi_result.nodes[0]
delhi_location = (delhi_node.lat, delhi_node.lon)

# Create shapely points for Kalka and Delhi
kalka_point = Point(kalka_location[::-1])  # (lon, lat)
delhi_point = Point(delhi_location[::-1])  # (lon, lat)

# Function to convert from degrees to meters
transformer = Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)
kalka_point_m = transform(transformer.transform, kalka_point)
delhi_point_m = transform(transformer.transform, delhi_point)

# Calculate the distance in meters
distance_m = kalka_point_m.distance(delhi_point_m)

# Convert the distance to kilometers
distance_km = distance_m / 1000

print(f"The distance between Kalka and Delhi is {distance_km:.2f} kilometers.")


The distance between Kalka and Delhi is 22027.77 kilometers.


In [3]:
import overpy
import geopandas as gpd
from shapely.geometry import Polygon

# Initialize the Overpass API
api = overpy.Overpass()

# Function to query and extract boundary data
def get_boundary(query):
    result = api.query(query)
    ways = result.ways
    if not ways:
        return None
    
    nodes = []
    for way in ways:
        for node in way.nodes:
            nodes.append((node.lon, node.lat))
    return Polygon(nodes)

# Query for Kalka boundary
kalka_query = """
area[name="Kalka"];
(way(area)[boundary=administrative];);
out body;
>;
out skel qt;
"""
kalka_boundary = get_boundary(kalka_query)

# Query for Delhi boundary
delhi_query = """
area[name="Delhi"];
(way(area)[boundary=administrative];);
out body;
>;
out skel qt;
"""
delhi_boundary = get_boundary(delhi_query)

# Create GeoDataFrames
gdf_kalka = gpd.GeoDataFrame([{'geometry': kalka_boundary, 'name': 'Kalka'}], crs="EPSG:4326")
gdf_delhi = gpd.GeoDataFrame([{'geometry': delhi_boundary, 'name': 'Delhi'}], crs="EPSG:4326")

# Save to shapefiles
gdf_kalka.to_file("kalka_boundary.shp")
gdf_delhi.to_file("delhi_boundary.shp")

print("Shapefiles for Kalka and Delhi boundaries have been saved.")


Shapefiles for Kalka and Delhi boundaries have been saved.
